### SVT Play 
version 0.101
* This [notebook](https://github.com/salgo60/open-data-examples/blob/master/SVTplay.ipynb)
   * Havnt found an offical API but [origin-www.svt.se/oppet-arkiv-api](https://origin-www.svt.se/oppet-arkiv-api)
     * SVT play id [1089610](https://origin-www.svt.se/oppet-arkiv-api/video/1089610) Operation Argus
     * [Latest](https://origin-www.svt.se/oppet-arkiv-api/latest?page=1&count=12pretty=true)
* we need an official copyright statement and API with the SVTprogramid 
   * id in this data looks like the SVT Öppet arkiv id

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2020-12-16 22:39:55.097533


In [2]:
import sys,json
import pandas as pd 
from tqdm import tqdm
import urllib3

http = urllib3.PoolManager()

count= 10
# https://origin-www.svt.se/oppet-arkiv-api/latest?page=1&count=1000&pretty=true
urlbase ="https://origin-www.svt.se/oppet-arkiv-api/latest?count=" + str(count) + "&page="

dftot = pd.DataFrame(columns=['id','programTitle','summary','materialLength','broadcastChannel',
                             'seasonNumber','episodeNumber',
                             'onlyAvailableInSweden','summary','dfparticipant'])
# number 5680
#for i in trange(0,570):
for i in tqdm(range(0,570)):    
    url = urlbase + str(i)
    dfparticipant = []
    #print (i, url)
    r = http.request('GET', url,headers={"Accept" : 'application/json'})
    data = json.loads(r.data)
    for e in data["entries"]:
        try:        
            id = e["id"]
        except:
            id = ""
        try:
            programTitle = e["programTitle"]
        except:
            programTitle = ""
        try:
            summary = e["summary"]
        except:
            summary =""
        try:
            materialLength = e["materialLength"]
        except:
            pass
        try:
            broadcastChannel = e["broadcastChannel"]
        except:
            pass
        try:
            seasonNumber = e["seasonNumber"]
        except:
            seasonNumber = ""
        try:
            episodeNumber = e["episodeNumber"]
        except:
            episodeNumber =""
        try:
            onlyAvailableInSweden = e["onlyAvailableInSweden"]
        except:
            onlyAvailableInSweden =""
        try:
            for p in e["tagList"]:
                
                if p["facet"] in "participantFacet":
                    try: 
                        if p['name'] is not None:
                            dfparticipant.append(p["name"])
                    except Exception as e: 
                        print (e)
        except:
            pass
        dftot = dftot.append({'id': id,'programTitle': programTitle,'materialLength': materialLength,
                             'broadcastChannel': broadcastChannel, 'seasonNumber':seasonNumber,
                             'episodeNumber': episodeNumber,'onlyAvailableInSweden': onlyAvailableInSweden,
                             'summary': summary, 'dfparticipant' : dfparticipant}, ignore_index=True)
dftot.info()

100%|██████████| 570/570 [03:22<00:00,  2.82it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682 entries, 0 to 5681
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     5682 non-null   object
 1   programTitle           5682 non-null   object
 2   summary                5682 non-null   object
 3   materialLength         5682 non-null   object
 4   broadcastChannel       5682 non-null   object
 5   seasonNumber           5682 non-null   object
 6   episodeNumber          5682 non-null   object
 7   onlyAvailableInSweden  5682 non-null   object
 8   summary                5682 non-null   object
 9   dfparticipant          5682 non-null   object
dtypes: object(10)
memory usage: 444.0+ KB


In [3]:
dftot.sample(100)

,id,programTitle,summary,materialLength,broadcastChannel,seasonNumber,episodeNumber,onlyAvailableInSweden,summary,dfparticipant
2058,1484528,Raskens,Om Fattigsverige i 1800-talets Småland med Sve...,3558,SVT 1,,2,True,Om Fattigsverige i 1800-talets Småland med Sve...,"[Kjell Bergqvist, Cecilia Nilsson, Göran Ragne..."
4224,1399333,Friidrott: DN-galan,TV-sporten visar ett sammandrag från årets stj...,6921,SVT 1,0,1,True,TV-sporten visar ett sammandrag från årets stj...,"[Birgitta Sandstedt, Vebjörn Rodal, Marlene Ot..."
4488,2547058,N P Möller fastighetsskötare,Fem händelser ur livet på Solgatan 16.,1843,SVT2,1,4,True,Fem händelser ur livet på Solgatan 16.,"[Leif GW Persson, Johanna Johnsson, Gary Engma..."
90,15983271,Kurt Olssons julkalender,Julkalendern 1990,858,SVT1,0,2,True,Julkalendern 1990,"[Lasse Brandeby, Hans Wiktorsson, Ulla Skoog, ..."
2305,3074277,Trafikmagasinet,Magasin om fordon och trafik.,3474,SVT1,,1,True,Magasin om fordon och trafik.,"[Hans Åke Nordström, Christer Glenning, Carl-I..."
...,...,...,...,...,...,...,...,...,...,...
3726,14106928,,Arkivinslag med Jonas Sjöstedt (V) 2011 - 2014.,623,SVT1,1,121,True,Arkivinslag med Jonas Sjöstedt (V) 2011 - 2014.,"[Anders Ygeman, Mats Hulth, Ulf Karlsson, Ingv..."
2061,1484471,Raskens,Om Fattigsverige i 1800-talets Småland med Sve...,3357,SVT 1,,5,True,Om Fattigsverige i 1800-talets Småland med Sve...,"[Kjell Bergqvist, Cecilia Nilsson, Göran Ragne..."
4295,1451622,Kungen och jag - och folket runt omkring,Tom Alandh följer kung Carl XVI Gustaf i 2 pro...,3456,SVT 2,0,1,True,Tom Alandh följer kung Carl XVI Gustaf i 2 pro...,"[Bengt Wadensjö, Claes-Bertil Ytterberg, Karl ..."
2825,1941650,Upp med händerna,Träningsprogram på tecknad svenska.,1121,SVT 2,0,1,True,Träningsprogram på tecknad svenska.,"[Ingrid Persson, Einar Linde, Gunnel Linde, Ma..."


In [4]:
dftot["broadcastChannel"].value_counts()[1:50]  


SVT 2              1828
SVT1                934
SVT2                318
                     17
NLOA                 11
SVT 1 och SVT 2      10
SVT  1                7
TV2                   5
SF journalen          4
SVT                   2
SVT 1, SVT 2          2
SVT1\n                2
SVT 1                 2
SVT  2                1
SVT24                 1
SVT 24                1
SVT                   1
SVT 2                 1
Name: broadcastChannel, dtype: int64

In [5]:
dftot["onlyAvailableInSweden"].value_counts()[1:50]  

False    101
Name: onlyAvailableInSweden, dtype: int64

In [6]:
dftot["programTitle"].value_counts()[1:20]  

Öppet arkiv: Sport                                     119
Öppet arkiv: fakta                                     107
Varuhuset                                               63
Öppet arkiv klipp: Nöje/underhållning                   62
Öppet arkiv: humor                                      61
Hem till byn                                            56
ÖPPET ARKIV: KULTUR                                     56
Goda grannar                                            49
Storstad                                                48
Öppet arkiv: barn                                       46
Tillbaka till Vintergatan                               39
Sunes jul                                               36
Lösa förbindelser                                       36
Melodifestivalen                                        35
Stackarsramsor                                          35
Spung 2.0                                               34
Pippi Långstrump                                        

In [7]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682 entries, 0 to 5681
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     5682 non-null   object
 1   programTitle           5682 non-null   object
 2   summary                5682 non-null   object
 3   materialLength         5682 non-null   object
 4   broadcastChannel       5682 non-null   object
 5   seasonNumber           5682 non-null   object
 6   episodeNumber          5682 non-null   object
 7   onlyAvailableInSweden  5682 non-null   object
 8   summary                5682 non-null   object
 9   dfparticipant          5682 non-null   object
dtypes: object(10)
memory usage: 444.0+ KB


In [11]:
#dftot.explode('dfparticipant')
participantSet = set()
for index, row in dftot.iterrows():
    if row['dfparticipant'] is not None:
        for p in row['dfparticipant']:
            participantSet.add(p)
    

In [14]:
print("Length: ", len(participantSet))

Length:  6451


In [17]:
print( list(participantSet)[1:30])

['Ulla Rodhe', 'The Latin Kings', 'Gun Holmqvist', 'Nils Lundgren', 'Carro Bergkvist', 'Stefan Comwell', 'Inga Ålenius', 'Halvard Jensen', 'Peje Emilsson', 'Kalle Westerdahl', 'Ittla Frodi', 'Tom Cruise', 'Walter Nicks', 'Karin Miller', 'Ulla Fornaeus', 'Kim Johansson', 'Tina Lenne', 'Lennart Holm', 'Ai', 'Elisabeth Aldén', 'Nils Fläcke', 'Antti Reini', 'Johanna Mork', 'Dan Lindhe', 'Hans-Elis Johansson', 'Rolf Lassgård', 'Rune Ek', 'J M Santana', 'Birgitta Dahl']
